# Supervised Feature Selection (Section 5.1)

This notebook recreates the supervised feature selection experiment on the Parkinsons dataset as described in Section 5.1 of the paper.

In [91]:
import sys
import os

# Add implementation/src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [92]:
import pandas as pd
import numpy as np
import os, sys
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from scipy.special import gamma,psi,digamma
from numpy import pi
import math
import os
from numpy import linalg as LA
import time


# Ross 2014 Estimators (Entropy & Mutual Information)

In [93]:
def mutual_information_continuous_discrete(x, y, k):
    """
    Calculates the mututal information between a continuous vector x and a
    disrete class vector y.
    This implementation can calculate the MI between the joint distribution of
    one or more continuous variables (X[:, 1:3]) with a discrete variable (y).
    Thanks to Adam Pocock, the author of the FEAST package for the idea.
    Brian C. Ross, 2014, PLOS ONE
    Mutual Information between Discrete and Continuous Data Sets
    """

    y = y.flatten()
    n = x.shape[0]
    classes = np.unique(y)
    knn = NearestNeighbors(n_neighbors=k)
    # distance to kth in-class neighbour
    d2k = np.empty(n)
    # number of points within each point's class
    Nx = []
    for yi in y:
        Nx.append(np.sum(y == yi))

    # find the distance of the kth in-class point
    for c in classes:
        mask = np.where(y == c)[0]
        knn.fit(x[mask, :])
        d2k[mask] = knn.kneighbors()[0][:, -1]

    # find the number of points within the distance of the kth in-class point
    knn.fit(x)
    m = knn.radius_neighbors(radius=d2k, return_distance=False)
    m = [i.shape[0] for i in m]

    # calculate MI based on Equation 2 in Ross 2014
    MI = psi(n) - np.mean(psi(Nx)) + psi(k) - np.mean(psi(m))
    return MI


def nearest_distances(X, k=1):
    """
    Returns the distance to the kth nearest neighbor for every point in X
    """

    knn = NearestNeighbors(n_neighbors=k, metric="chebyshev")
    knn.fit(X)
    # the first nearest neighbor is itself
    d, _ = knn.kneighbors(X)
    # returns the distance to the kth nearest neighbor
    return d[:, -1]


def entropy(X, k=1):
    """
    Returns the entropy of the X.
    Written by Gael Varoquaux:
    https://gist.github.com/GaelVaroquaux/ead9898bd3c973c40429
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The data the entropy of which is computed
    k : int, optional
        number of nearest neighbors for density estimation
    References
    ----------
    Kozachenko, L. F. & Leonenko, N. N. 1987 Sample estimate of entropy
    of a random vector. Probl. Inf. Transm. 23, 95-101.
    See also: Evans, D. 2008 A computationally efficient estimator for
    mutual information, Proc. R. Soc. A 464 (2093), 1203-1215.
    and:
    Kraskov A, Stogbauer H, Grassberger P. (2004). Estimating mutual
    information. Phys Rev E 69(6 Pt 2):066138.
    F. Perez-Cruz, (2008). Estimation of Information Theoretic Measures
    for Continuous Random Variables. Advances in Neural Information
    Processing Systems 21 (NIPS). Vancouver (Canada), December.
    return d*mean(log(r))+log(volume_unit_ball)+log(n-1)-log(k)
    """

    # Distance to kth nearest neighbor
    r = nearest_distances(X, k)
    n, d = X.shape
    volume_unit_ball = (np.pi ** (0.5 * d)) / gamma(0.5 * d + 1)
    return (
        d * np.mean(np.log(r + np.finfo(X.dtype).eps))
        + np.log(volume_unit_ball)
        + psi(n)
        - psi(k)
    )

# Calculate $\rho$ and $Q$

In [94]:
def calculate_rho_and_Q(seed):
    df = pd.read_csv("../data/parkinsons.data")
    a = df.loc[:, df.columns != "status"].values[:, 1:]
    b = df.loc[:, "status"].values
    features = df.loc[:, df.columns != "status"].values[:, 1:].reshape(195, 22)
    labels = df.loc[:, "status"].values.reshape(195, 1)
    df = np.hstack((features, labels))
    df = pd.DataFrame(
        df,
        columns=[
            "MDVP:Fo(Hz)",
            "MDVP:Fhi(Hz)",
            "MDVP:Flo(Hz)",
            "MDVP:Jitter(%)",
            "MDVP:Jitter(Abs)",
            "MDVP:RAP",
            "MDVP:PPQ",
            "Jitter:DDP",
            "MDVP:Shimmer",
            "MDVP:Shimmer(dB)",
            "Shimmer:APQ3",
            "Shimmer:APQ5",
            "MDVP:APQ",
            "Shimmer:DDA",
            "NHR",
            "HNR",
            "RPDE",
            "DFA",
            "spread1",
            "spread2",
            "D2",
            "PPE",
            "status",
        ],
    )
    print(labels.shape, features.shape)
    p_ig = mutual_info_classif(features, labels.reshape(-1), random_state=seed)  # 1x22

    # p_fs = fisher_score.fisher_score(features,labels)
    Q = np.zeros((22, 22))
    k = 5  # 7
    for i in range(22):
        for j in range(22):
            H = entropy(
                np.array([a[:, i]], dtype="float64").reshape(195, 1), k
            ) + entropy(np.array([a[:, j]], dtype="float64").reshape(195, 1), k)
            # print(np.array(features[:,i]).reshape(1,195).shape)
            # print(np.vstack((features[:,i],features[:,j])).T.shape)
            tmp1 = mutual_information_continuous_discrete(
                np.array(features[:, i]).reshape(1, 195).T, labels, k
            )
            tmp2 = mutual_information_continuous_discrete(
                np.array(features[:, j]).reshape(1, 195).T, labels, k
            )
            # print(tmp1)
            # print(np.vstack((features[:,i],features[:,j])).T)
            tmp3 = mutual_information_continuous_discrete(
                np.vstack((features[:, i], features[:, j])).T, labels, k
            )
            tmp = tmp1 + tmp2 - tmp3
            # print(tmp1+tmp2-tmp3)
            Q[i, j] = max(0, tmp / H)
    
    w, _ = LA.eig(Q)
    w
    xi = -min(0,min(w))
    Q = xi * np.eye(22) + Q
    return np.array(p_ig), Q, features, labels

In [95]:
def make_objective(Q, rho):
    """
    return f(w) = (w^T Q w) / (rho^T w)
    """
    def fractional_objective(w):
        num = w.T @ Q @ w
        den = rho.T @ w
        return num / den
    return fractional_objective

In [96]:
from scipy.optimize import Bounds, LinearConstraint
from algorithms import Projector, GDA
import autograd.numpy as anp

results = []

for seed in range(1, 100, 10):
    rho, Q, X, y = calculate_rho_and_Q(seed)


    def f(x):
        return (anp.dot(anp.dot(x,Q),x.T))/(anp.dot(rho,x.T))

    
    n_features = len(rho)

    bounds = Bounds([0] * n_features, [np.inf] * n_features)
    constraints = [
        LinearConstraint(np.ones((1, n_features)), [1], [1])  # sum(w) = 1
    ]
    projector = Projector(bounds, constraints)
    f = make_objective(Q, rho)
    gda = GDA(function=f, projector=projector)
    w_init = np.ones(n_features) / n_features

    from scipy.optimize import minimize

    start = time.time()
    res_scipy = minimize(fun=f, x0=w_init, method='SLSQP', bounds=bounds, constraints=constraints, tol=1e-12)
    res_scipy.f_opt = f(res_scipy.x)
    time_scipy = time.time() - start

    start = time.time()
    res_gda = gda.solve(w_init, tol=1e-12)
    time_gda = time.time() - start

    results.append((seed, res_gda, time_gda, res_scipy, time_scipy))

(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)
(195, 1) (195, 22)


In [97]:
# Draw a table comparing the two results
title = "Comparing Scipy vs GDA"

print(f"+{'-' * 100}+")
print(f"|{title:^100}|")
print(f"+{'-' * 100}+")
# Print a table of results with f opt, time, iters
print(f"|{'seed':^7} | {'Algorithm GDA (proposed)':^43} | {'Scipy':^43} |")
print(f"+{'-' * 8}+{'-' * 45}+{'-' * 45}+")
print(f"|{'':>7} | {'f_opt':^16} | {'time (ms)':^11} | {'iters':^10} | {'f_opt':^16} | {'time (ms)':^11} | {'iters':^10} |")
print(f"+{'-' * 8}+{'-' * 18}+{'-' * 13}+{'-' * 12}+{'-' * 18}+{'-' * 13}+{'-' * 12}+")
for n, res_gda, time_gda, res_scipy, time_scipy in results:
    print(f"| {n:6d} | {res_gda.f_opt:16.6f} | {time_gda:11.6f} | {len(res_gda.x_history):10d} | {res_scipy.f_opt:16.6f} | {time_scipy:11.6f} | {(res_scipy.nit):10d} |")
print(f"+{'-' * 8}+{'-' * 18}+{'-' * 13}+{'-' * 12}+{'-' * 18}+{'-' * 13}+{'-' * 12}+")

+----------------------------------------------------------------------------------------------------+
|                                       Comparing Scipy vs GDA                                       |
+----------------------------------------------------------------------------------------------------+
| seed   |          Algorithm GDA (proposed)           |                    Scipy                    |
+--------+---------------------------------------------+---------------------------------------------+
|        |      f_opt       |  time (ms)  |   iters    |      f_opt       |  time (ms)  |   iters    |
+--------+------------------+-------------+------------+------------------+-------------+------------+
|      1 |         0.152478 |    0.018614 |         32 |         0.152478 |    0.006164 |         19 |
|     11 |         0.153480 |    0.096673 |         51 |         0.153480 |    0.030229 |         15 |
|     21 |         0.153834 |    0.015980 |         31 |         0.153834

Có thể thấy, mỗi lần chạy có kết quả khác nhau, do mutual_info_classif tạo random noise. Do đó implementation của paper thực sự không đáng tin cậy.